In [1]:
import argparse
import time
import numpy as np

from queue import Queue

import brainflow
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes, AggOperations

from pylsl import StreamInfo, StreamOutlet, local_clock

In [2]:
def channel_select(board, board_id, data_type): 
    switcher = { 
        'EXG': board.get_exg_channels(board_id),
        # can add more
    } 
 
    return switcher.get(data_type, "error")

In [3]:
params = BrainFlowInputParams();
#params.ip_address = "192.168.4.1"; # for direct connect
params.ip_address = "192.168.0.102";
params.ip_port = 6622;
params.timeout = 0;
board_id = BoardIds.CYTON_DAISY_WIFI_BOARD.value;
board = BoardShim(board_id,params);
print("Sampling Rate: ", board.get_sampling_rate(BoardIds.CYTON_DAISY_WIFI_BOARD.value));
print("Device name: ", board.get_device_name(BoardIds.CYTON_DAISY_WIFI_BOARD.value));
#print(board.get_sampling_rate(BoardIds.CYTON_DAISY_WIFI_BOARD.value));
#print("Is prepared: ", board.is_prepared());

Sampling Rate:  1000
Device name:  CytonDaisyWifi


In [4]:
# LSL initialization
eeg_channels = BoardShim.get_eeg_channels(board_id)
n_channels = len(eeg_channels)
srate = board.get_sampling_rate(board.board_id)
info = StreamInfo('OpenBCIEEG1', 'EXG', n_channels, srate, 'double64', 'OpenBCItestEEG1')
outlet = StreamOutlet(info)
fw_delay = 0

In [5]:
board.prepare_session()

# send commands to the board for every channel. Cyton has 8 Channels. Here, we turn off every channel except for 1 and 8.
# This is here for testing purposes.
#board.config_board("x1000110X") #Lower the gain to 1x on channel 1
#board.config_board("x1061000X")
#board.config_board("x2161000X")
#board.config_board("x3161000X")
#board.config_board("x4161000X")
#board.config_board("x5161000X")
#board.config_board("x6161000X")
#board.config_board("x7161000X")
#board.config_board("x8060110X")

# start stream
board.start_stream(45000)
time.sleep(1)

start_time = local_clock()
sent_samples = 0
queue = Queue(maxsize = 5*srate)
chans = channel_select(board, board.board_id, 'EXG')

In [ ]:
# read data with brainflow and send it via LSL
print("Now sending data...")
first = True
while True:
    data = board.get_board_data()[chans]
    # Dark Mode
    if first:
        board.config_board("P");
        first = False

    for i in range(len(data[0])):
        queue.put(data[:,i].tolist())
    elapsed_time = local_clock() - start_time
    required_samples = int(srate * elapsed_time) - sent_samples
    if required_samples > 0 and queue.qsize() >= required_samples:    
        mychunk = []

        for i in range(required_samples):
            mychunk.append(queue.get())
        stamp = local_clock() - fw_delay 
        outlet.push_chunk(mychunk)
        #print(mychunk[0])
        sent_samples += required_samples
    time.sleep(1)

Now sending data...
